In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import time
from datetime import date

In [2]:
base_url = catalog.load('params:oai_fetch_options.base_url')
context = catalog.load('params:oai_fetch_options.context')
env = 'dev'

                    INFO     Loading data from params:oai_fetch_options.base_url                ]8;id=396637;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=242555;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:oai_fetch_options.context                 ]8;id=163840;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=375477;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [7]:
base_url

'https://ri.conicet.gov.ar/oai/'

In [16]:
def get_oai_records(base_url):
    start_time = time.time()

    response = requests.get(base_url)
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Sleeping for {elapsed_time:.2f} seconds")
    time.sleep(elapsed_time)

    if response.status_code == 200:
        return response
    else:
        print(f"Error: {response.status_code}")
        return None

def parse_oai_response(xml_data):
    root = ET.fromstring(xml_data)
    namespaces = {'oai': 'http://www.openarchives.org/OAI/2.0/',
                  'dc': 'http://purl.org/dc/elements/1.1/'}

    records = []
    for record in root.findall('.//oai:record', namespaces):
        metadata = record.find('.//oai:metadata', namespaces)
        if metadata is not None:
            record_dict = {}
            for elem in metadata.findall('.//dc:*', namespaces):
                tag = elem.tag.split('}')[-1]
                if tag in record_dict:
                    record_dict[tag].append(elem.text)
                else:
                    record_dict[tag] = [elem.text]
            records.append(record_dict)

    return records

def fetch_oai_items(base_url, context, env):
    
    resumption_token = '0'
    params = f'/{context}?verb=ListRecords&resumptionToken=oai_dc////'
    url = base_url + params + str(resumption_token)
    items = []
    resumption_token = 0
    url = base_url + params + str(resumption_token)

    response = get_oai_records(url)
    records = parse_oai_response(response.text)
    record_size = len(records) 
    items.extend(records)

    iteration_limit = 2
    iteration_count = 0

    while record_size > 0:

        if env == 'dev' and iteration_count >= iteration_limit:
            break

        record_size = len(records)
        resumption_token += 100
        url = base_url + params + str(resumption_token)
        print(url)
        response = get_oai_records(url)
        records = parse_oai_response(response.text)
        items.extend(records)
        iteration_count += 1

    df = pd.DataFrame(items)
    return df


In [ ]:
df = fetch_oai_items(base_url, context, env)

Sleeping for 0.12 seconds
https://ri.conicet.gov.ar/oai//request?verb=ListRecords&metadataPrefix=oai_dc&resumptionToken=0////0


In [5]:
df

""
